##### CURSO INF0083 - TECNOLOGIAS AVANÇADAS EM IA  
##### DISCIPLINA INF0084 - Sistemas Inteligentes e Técnicas Avançadas em IA  
##### Tarefa 02: RAG Multimodal
##### Alunos: Fabio Grassiotto e Wandemberg Gibaut

Imports

In [ ]:
from bs4 import BeautifulSoup
import os
import glob
import requests
import unicodedata
import json
from functions import *
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from torch import cat, save
from io import BytesIO
from PIL import Image

Leitura dos arquivos html e criação das listas de textos e imagens.

In [2]:
# Lista de arquivos no diretórion "raw"
html_files = glob.glob(os.path.join('raw', '*.html'))

texts = []
images = []
images_list = []

def download_image(url):
    response = requests.get(url)
    if response.status_code == 200:
        img = Image.open(BytesIO(response.content))
        return img
    else:
        print(f"Failed to download {url}")
        return None

# Remoção de caracteres com Unicode
def normalize_text(text):
    return unicodedata.normalize('NFKD', text)
    
# Popula listas
for file in html_files:
    with open(file, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'html.parser')
        
        # Extração do texto
        text = soup.get_text()
        text = normalize_text(text)
        text_lines = text.split('\n')
        texts.extend([line.strip() for line in text_lines if line.strip()])        
        
        # Criação de objetos para imagens
        for img in soup.find_all('img'):
            img_src = img.get('src')
            images_list.append(img_src)
            pil_image = download_image(img_src)
            if pil_image:
                images.append(pil_image)

Inspeção dos resultados

In [3]:
texts

['Multimodal Models — LLMs that can see and hear',
 'Multimodal Models — LLMs that can see and hear',
 'An introduction with example Python code',
 'Multimodal Models — LLMs That Can See and HearAn introduction with example Python codeThis is the first post in a larger series on Multimodal AI. A Multimodal Model (MM) is an AI system capable of processing or generating multiple data modalities (e.g., text, image, audio, video). In this article, I will discuss a particular type of MM that builds on top of a large language model (LLM). I’ll start with a high-level overview of such models and then share example code for using LLaMA 3.2 Vision to perform various image-to-text tasks.Photo by Sincerely Media on UnsplashLarge language models (LLMs) have marked a fundamental shift in AI research and development. However, despite their broader impacts, they are still fundamentally limited.Namely, LLMs can only process and generate text, making them blind to other modalities such as images, video

In [4]:
images

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=800x600>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=800x262>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=800x322>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=800x378>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=800x1016>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=800x800>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=800x1308>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=800x450>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=800x439>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=800x586>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=800x435>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=800x423>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=366x342>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=366x273>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=288x256>,
 <PIL.PngImagePlugi

Carrega o modelo pré-treinado do CLIP e o processador do repositório da OpenAI:

In [5]:
model_name = "openai/clip-vit-base-patch16"
model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)

Geração dos embeddings para os textos e imagens:

In [6]:
text_inputs = processor(text=texts, return_tensors="pt", padding=True, truncation=True)
text_embeddings = model.get_text_features(**text_inputs)

In [7]:
inputs = processor(images=images, return_tensors="pt", padding=True)
image_embeddings = model.get_image_features(**inputs)

Salva os embeddings e listas nos arquivos JSON na pasta "data"

In [8]:
save(text_embeddings, 'data/text_embeddings.pt')
save(image_embeddings, 'data/image_embeddings.pt')

In [9]:
def save_to_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

save_to_json(texts, 'data/text_content.json')
save_to_json(images_list, 'data/image_content.json')